# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [2]:
licenca ="default"


ampl = tools.ampl_notebook(
    modules=["highs", "gcg", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

In [3]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [4]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [5]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

In [6]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [7]:
df_process['tempo_espera_dias'] = (df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16')


## Caixa padrão

In [8]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM_edit.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [9]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')


In [10]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [11]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [12]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [13]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,1,inf,1865_6.0,26_CX_5,3092300,98,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,1,inf,1865_6.0,26_CX_5,3321500,98,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,1,inf,1865_6.0,26_CX_5,3537200,98,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,1,3,inf,2489_674.0,116_CX_6,233624,37,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,1,inf,2988_451.0,16_CX_2,775500,37,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,27778950.0,83393894.4,75538008.0


In [14]:
df_process_capacitado.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'volume_util_caixa_Caixa 1', 'volume_util_caixa_Caixa 2',
       'volume_util_caixa_Caixa 3'],
      dtype='object')

In [15]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [16]:
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [17]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,inf,1865_6.0,26_CX_5,3092300,98,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,inf,1865_6.0,26_CX_5,3321500,98,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,inf,1865_6.0,26_CX_5,3537200,98,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,inf,2489_674.0,116_CX_6,233624,37,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,inf,2988_451.0,16_CX_2,775500,37,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,4.000000e+10,1863_2.0,48_EN_2,3269280,0,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,4.000000e+10,1863_2.0,48_EN_2,4037285,0,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,4.000000e+10,1863_2.0,48_EN_2,3231360,0,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,4.000000e+10,713_275.0,48_EN_2,4235707,0,27778950.0,83393894.4,75538008.0,75538008.0,512


In [18]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

## Rotas

In [19]:
df_rotas = df_process_capacitado[['Rota', 'Capacidade_rota']].drop_duplicates()
df_rotas = df_rotas.sort_values(by='Rota').reset_index(drop=True)
df_rotas

,Rota,Capacidade_rota
0,1,inf
1,2,2.720000e+10
2,3,inf
3,4,inf
4,5,2.720000e+10
5,6,2.720000e+10
6,7,4.000000e+10
7,8,4.000000e+10
8,9,2.720000e+10
9,10,2.720000e+10


# 2- Modelagem

## 2.1- Modelo exato

Vamos incluir a restrição de de tipo de buffer

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, então temos:

$$
\begin{aligned}
\text{maximizar} \quad &\sum_{i = 1}^{n} \sum_{k = 1}^{r}  z_{i, k}\\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} \leq  d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\sum_{j = 1}^{m} y_{j k}*cv_{k} \leq Vr_{k} \quad \forall k \in \{1,...,r\}\\
&\sum_{j = 1}^{m} \xi_{i j k} \leq M z_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\sum_{j = 1}^{m} \xi_{i j k} \geq z_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&y_{j, k} \geq y_{j+1, k} \quad \forall j \in \{1,...,m-1\} , k \in \{1,...,r\} \\
&z_{i, k} \geq z_{i+1, k} \quad \forall i \in \{1,...,n-1\} , k \in \{1,...,r\} \\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&z_{i k} \in \{0,1\} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja 
 
 - $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.
 - $z_{i, k}$ = 1, se o item $i$ da loja $k$ foi alocado em alguma caixa $j$; 0, caso contrário.


In [55]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)
param M;  # Big-M

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)
param Vr {K} >= 0; # volume da rota da chave k

param capac_vol_cx := min {k in K} cv [k];

param maxVal_vol := sum {i in I, k in K} d [i,k] *v [i,k];

param maxbins := ceil(maxVal_vol / capac_vol_cx);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a loja k
var z {I,K} binary; # 1 se o item i da loja k é alocado em alguma caixa j

maximize FO:  sum {i in I, k in K, j in J} e[i,j,k] * inner[i,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k];

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k];

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k];

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] <= d[i,k];

# Limite superior para o somatório dos volumes das caixas em uma rota
s.t. R5_volume_rota_superior {k in K}:
    sum {j in J} y[j,k] * cv[k] <= Vr[k];

# Restrição Superior
s.t. R6_upper_bound_z {i in I, k in K}:
    sum {j in J} e[i, j, k] <= M * z[i, k];

# Restrição Inferior
s.t. R7_lower_bound_z {i in I, k in K}:
    sum {j in J} e[i, j, k] >= z[i, k];

# Predecedência entre as caixas
s.t. R8_predecessor_caixas {j in 1..maxbins-1, k in K}:
   y[j,k] >= y[j+1,k];

# Predecedência entre os itens
s.t. R9_predecessor_itens {i in 1..n-1, k in K}:
   z[i,k] >= z[i+1,k];


### 2.1.1- Situação 1

In [56]:
df_process_situacao1 = df_process_capacitado.query("chave_loja_buffer_onda in ('48_EN_4', '8_EN_3', '10_EN_1')")
df_process_situacao1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
33706,33706,1797,2023-10-18,595814344,505,206,40,0.237,3,CARTON_M,...,2.720000e+10,361_9.0,8_EN_3,4161200,35,27778950.0,83393894.4,75538008.0,75538008.0,512
33748,33748,5649,2023-10-26,873646759,450,279,22,0.207,2,CARTON_M,...,2.720000e+10,2096_578.0,8_EN_3,2762100,27,27778950.0,83393894.4,75538008.0,75538008.0,512
33749,33749,7066,2023-10-26,873646741,450,293,22,0.207,5,CARTON_M,...,2.720000e+10,2096_578.0,8_EN_3,2900700,27,27778950.0,83393894.4,75538008.0,75538008.0,512
33750,33750,10193,2023-10-26,873646732,450,279,21,0.220,8,CARTON_M,...,2.720000e+10,2096_578.0,8_EN_3,2636550,27,27778950.0,83393894.4,75538008.0,75538008.0,512
33751,33751,10775,2023-10-26,873646716,440,306,23,0.233,2,CARTON_M,...,2.720000e+10,2096_578.0,8_EN_3,3096720,27,27778950.0,83393894.4,75538008.0,75538008.0,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59452,59452,10640,2023-11-21,841265687,411,300,63,0.615,1,CARTON_M,...,2.720000e+10,192_9.0,8_EN_3,7767900,1,27778950.0,83393894.4,75538008.0,75538008.0,512
59453,59453,10428,2023-11-21,549983673,363,238,16,0.232,1,CARTON_M,...,2.720000e+10,233_3.0,8_EN_3,1382304,1,27778950.0,83393894.4,75538008.0,75538008.0,512
59454,59454,628,2023-11-21,858645213,442,215,18,0.253,3,CARTON_M,...,2.720000e+10,571_54.0,8_EN_3,1710540,1,27778950.0,83393894.4,75538008.0,75538008.0,512
59455,59455,4260,2023-11-21,858645221,461,248,21,0.220,6,CARTON_M,...,2.720000e+10,571_54.0,8_EN_3,2400888,1,27778950.0,83393894.4,75538008.0,75538008.0,512


In [57]:
df_process_situacao1['Peças_max'].unique()

array([512], dtype=int64)

In [58]:
def matriz_ik(df,variavel_ik):
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
    df['item_loja'] = df.groupby('chave_loja_buffer_onda').cumcount() + 1

    df_matriz_peso = df.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

    df_matriz_peso.columns.name = None

    return df_matriz_peso

In [59]:
def matriz_k(df, variavel_k):
    df_caixas = df.pivot_table(index='chave_loja_buffer_onda',  values= variavel_k, fill_value=0).reset_index()

    df_caixas.columns.name = None

    return df_caixas

In [60]:
df_matriz_peso = matriz_ik(df_process_situacao1,'Peso_unit')

df_matriz_volume = matriz_ik(df_process_situacao1,'Volume_unit')

df_matriz_inner = matriz_ik(df_process_situacao1,'Inner')

df_matriz_pecas = matriz_ik(df_process_situacao1,'Peças')

In [61]:
df_matriz_pecas

,item_loja,10_EN_1,48_EN_4,8_EN_3
0,1,4.0,12.0,3.0
1,2,6.0,2.0,2.0
2,3,1.0,1.0,5.0
3,4,1.0,2.0,8.0
4,5,37.0,6.0,2.0
...,...,...,...,...
443,444,0.0,0.0,1.0
444,445,0.0,0.0,1.0
445,446,0.0,0.0,3.0
446,447,0.0,0.0,6.0


In [62]:
df_matriz_caixa_capacidade_pecas = matriz_k(df_process_situacao1, 'Peças_max')

df_matriz_caixa_capacidade_volume = matriz_k(df_process_situacao1, 'volume_util_padrao')

In [63]:
df_process_situacao1.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max',
       'item_loja'],
      dtype='object')

In [64]:
df_matriz_caixa_capacidade_rota = matriz_k(df_process_situacao1, 'Capacidade_rota')
df_matriz_caixa_capacidade_rota

,chave_loja_buffer_onda,Capacidade_rota
0,10_EN_1,4.000000e+10
1,48_EN_4,4.000000e+10
2,8_EN_3,2.720000e+10


In [65]:
df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_EN_1,75538008.0
1,48_EN_4,75538008.0
2,8_EN_3,75538008.0


Características das peças

In [66]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

Características da caixa

In [67]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas
M = 100e3
Vr = df_matriz_caixa_capacidade_rota

#### Parâmetros

In [68]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["M"] = M
ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')
Vr_ = Vr.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}
ampl.param["Vr"] = {(k): Vr_.iloc[k - 1] for k in range(1, len(Vr_) + 1)}

In [69]:
d

,10_EN_1,48_EN_4,8_EN_3
0,4.0,12.0,3.0
1,6.0,2.0,2.0
2,1.0,1.0,5.0
3,1.0,2.0,8.0
4,37.0,6.0,2.0
...,...,...,...
443,0.0,0.0,1.0
444,0.0,0.0,1.0
445,0.0,0.0,3.0
446,0.0,0.0,6.0


In [70]:
ampl.getValue('maxbins')

236

### 2.2.2- Solvers

In [71]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+05]
  RHS    [1e+00, 4e+01]
Presolving model
7181 rows, 318528 cols, 1610170 nonzeros  0s
7181 rows, 244268 cols, 1461650 nonzeros  1s
6233 rows, 243636 cols, 1460017 nonzeros  9s
Objective function is integral with scale 1

Solving MIP model with:
   6233 rows
   243636 cols (63332 binary, 180304 integer, 0 implied int., 0 continuous)
   1460017 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   854111          -inf                 inf        0      0      0         0    11.4s
 R       0       0         0   0.00%   4032     

In [72]:
%%ampl_eval
display FO, y;

FO = 4032

y [*,*]
:     1   2   3    :=
1     1   1   1
2     1   1   1
3     1   1   1
4     1   1   1
5     1   1   1
6     1   1   1
7     1   1   1
8     1   1   1
9     1   1   1
10    1   1   1
11    1   1   1
12    1   1   1
13    1   1   1
14    1   1   1
15    1   1   1
16    1   1   1
17    1   1   1
18    1   1   1
19    1   1   1
20    1   1   1
21    1   1   1
22    1   1   1
23    1   1   1
24    1   1   1
25    1   1   1
26    1   1   1
27    1   1   1
28    1   1   1
29    1   1   1
30    1   1   1
31    1   1   1
32    1   1   1
33    1   1   1
34    1   1   1
35    1   1   1
36    1   1   1
37    1   1   1
38    1   1   1
39    1   1   1
40    1   1   1
41    1   1   1
42    1   1   1
43    1   1   1
44    1   1   1
45    1   1   1
46    1   1   1
47    1   1   1
48    1   1   1
49    1   1   1
50    1   1   1
51    1   1   1
52    1   1   1
53    1   1   1
54    1   1   1
55    1   1   1
56    1   1   1
57    1   1   1
58    1   1   1
59    1   1   1
60    1   1   1

In [73]:
%%ampl_eval
display e;

e [*,*,1]
:       1          2         3   4        5         6   7   8   9   10  11  12 :=
1         0    0             0   0    0             0   0   0   0    0   0   0
2         0    0             0   0    0             0   0   0   0    0   0   0
3         0    0             0   0    0             0   0   0   0    0   0   0
4         0    0             0   0    0             0   0   0   0    0   0   0
5         0    0             0   0    0             0   0   0   0    0   0   0
6         0    0             0   0    0             0   0   0   0    0   0   0
7         0    0             0   0    0             0   0   0   0    0   0   0
8         0    0             0   0    0             0   0   0   0    0   0   0
9         0    0             0   0    0             0   0   0   0    0   0   0
10        0    0             0   0    0             0   0   0   0    0   0   0
11        0    0             0   0    0             0   0   0   0    0   0   0
12        0    0             0   0    0

In [74]:
inner

,item_loja,10_EN_1,48_EN_4,8_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
443,444,0.0,0.0,1.0
444,445,0.0,0.0,1.0
445,446,0.0,0.0,1.0
446,447,0.0,0.0,1.0


In [75]:
ampl.getVariable('z').getValues().toPandas()

z.val
index0 index1       
1      1           1
       2           1
       3           1
2      1           1
       2           1
...              ...
447    2           0
       3           1
448    1           0
       2           0
       3           1

[1344 rows x 1 columns]

In [76]:
z_s1_highs = ampl.getVariable('z').getValues().toPandas()
z_s1_highs = z_s1_highs.reset_index()
z_s1_highs.columns = ['item_loja', 'chave_loja_buffer_onda', 'z']
z_s1_highs = z_s1_highs.query('z >0')
z_s1_highs


,item_loja,chave_loja_buffer_onda,z
0,1,1,1
1,1,2,1
2,1,3,1
3,2,1,1
4,2,2,1
...,...,...,...
1331,444,3,1
1334,445,3,1
1337,446,3,1
1340,447,3,1


In [77]:
e_s1_highs = ampl.getVariable('e').getValues().toPandas()
e_s1_highs = e_s1_highs.reset_index()
e_s1_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_highs = e_s1_highs.query('valor_multiplo >0')
e_s1_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
374,1,125,3,1.0
400,1,134,2,11.0
502,1,168,2,1.0
561,1,188,1,4.0
632,1,211,3,2.0
...,...,...,...,...
316124,447,119,3,1.0
316256,447,163,3,2.0
316379,447,204,3,1.0
316490,448,5,3,1.0


In [78]:
e_s1_highs.query('valor_multiplo >1')

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
400,1,134,2,11.0
561,1,188,1,4.0
632,1,211,3,2.0
1132,2,142,2,2.0
1134,2,143,1,5.0
...,...,...,...,...
312800,442,191,3,2.0
315446,446,129,3,3.0
316046,447,93,3,2.0
316256,447,163,3,2.0


In [79]:
df_agrupamento_loja1_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s1 = df_agrupamento_loja1_highs_s1.reset_index()
df_agrupamento_loja1_highs_s1.columns.name = None
df_agrupamento_loja1_highs_s1 # as colunas são as caixas

,item_loja,1,2,3,10,11,12,13,15,16,...,221,222,224,226,228,229,230,231,232,235
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,413,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413,414,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
414,415,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415,416,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao1.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [81]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [82]:
len(vetores[f"{w.columns[0]}"])

223

In [83]:
len(vetores[f"{w.columns[1]}"])

357

In [84]:
len(vetores[f"{w.columns[2]}"])

448

In [86]:
df_agrupamento_loja1_highs_s1['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s1

ValueError: Length of values (223) does not match length of index (417)